In [1]:
import ray
import time

In [2]:
def normal_function():
    return 1

@ray.remote
def my_function():
    return 1

In [10]:
%%timeit -n 5
obj_ref = my_function.remote()
ray.get(obj_ref)

1.32 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [11]:
%%timeit -n 5
o = normal_function()
o

140 ns ± 139 ns per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [17]:
@ray.remote
def slow_function():
    time.sleep(5)
    return 1

In [49]:
obj_refs = [slow_function.remote() for _ in range(24)]
print(obj_refs)

for obj_ref in obj_refs:
    print(ray.get(obj_ref))

[ObjectRef(7a8864067fa5553fffffffffffffffffffffffff0100000001000000), ObjectRef(ab500821de3397dbffffffffffffffffffffffff0100000001000000), ObjectRef(0190e882c39dde83ffffffffffffffffffffffff0100000001000000), ObjectRef(043b57d82bcae36cffffffffffffffffffffffff0100000001000000), ObjectRef(2293a32bde7452deffffffffffffffffffffffff0100000001000000), ObjectRef(947cf4274b34a51dffffffffffffffffffffffff0100000001000000), ObjectRef(fd51e76d364914edffffffffffffffffffffffff0100000001000000), ObjectRef(37184d4ca2b304b1ffffffffffffffffffffffff0100000001000000), ObjectRef(ab48262718bdf969ffffffffffffffffffffffff0100000001000000), ObjectRef(9009382f3d9fce80ffffffffffffffffffffffff0100000001000000), ObjectRef(c419c1bd6fc88196ffffffffffffffffffffffff0100000001000000), ObjectRef(50dbd2fb2b3daa94ffffffffffffffffffffffff0100000001000000), ObjectRef(e021565934be4929ffffffffffffffffffffffff0100000001000000), ObjectRef(dcebe4033e16fde4ffffffffffffffffffffffff0100000001000000), ObjectRef(4b12b3aaf88b06fffffffff

: 

In [18]:
@ray.remote
def function_with_an_argument(value):
    return value + 1

obj_ref1 = slow_function.remote()
assert ray.get(obj_ref1) == 1

# You can pass an object ref as an argument to another Ray remote function.
obj_ref2 = function_with_an_argument.remote(obj_ref1)
assert ray.get(obj_ref2) == 2

In [28]:
ready_refs, remaining_refs = ray.wait(obj_refs, num_returns=5, timeout=None)
print(len(obj_refs))
print(len(ready_refs), len(remaining_refs))
ready_refs, remaining_refs

10
5 5


([ObjectRef(6efee821243c7757ffffffffffffffffffffffff0100000001000000),
  ObjectRef(859398ca83c3d05affffffffffffffffffffffff0100000001000000),
  ObjectRef(b09690aaeb916171ffffffffffffffffffffffff0100000001000000),
  ObjectRef(81a37bdbafbadd26ffffffffffffffffffffffff0100000001000000),
  ObjectRef(215ddf64c4c571b9ffffffffffffffffffffffff0100000001000000)],
 [ObjectRef(5c6a7a80be26e789ffffffffffffffffffffffff0100000001000000),
  ObjectRef(83105f4bf619a6f9ffffffffffffffffffffffff0100000001000000),
  ObjectRef(ba5afcb0f9a96efeffffffffffffffffffffffff0100000001000000),
  ObjectRef(eeb91e012ec4396fffffffffffffffffffffffff0100000001000000),
  ObjectRef(f95c61a87fc9f9beffffffffffffffffffffffff0100000001000000)])

In [48]:
@ray.remote
class Counter(object):
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1
        return self.value

# Create an actor from this class.
counter = Counter.remote()
counters = [Counter.remote() for _ in range(20)]

# Increment each Counter once and get the results. These tasks all happen in
# parallel.
results = ray.get([c.increment.remote() for c in counters])
print(results)  # prints [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# Increment the first Counter five times. These tasks are executed serially
# and share state.
results = ray.get([counters[0].increment.remote() for _ in range(5)])
print(results)  # prints [2, 3, 4, 5, 6]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2, 3, 4, 5, 6]


In [40]:
counter.increment.remote()

ObjectRef(5711e88412104e5a4c66c8baf2886b50a70289350100000001000000)

In [44]:
y = 1
y = ray.put(y)
y

ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000003000000)

In [46]:
ray.get(y)

1

In [47]:
y

ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000003000000)